<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Date completed: 30 October 2024


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [51]:
!pip3 install folium
!pip3 install wget
print("Installation complete.")

Installation complete.


In [52]:
import folium
import wget
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
print("All libraries have been imported.")

All libraries have been imported.


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [53]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [54]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [55]:
# Start location is NASA Johnson Space Center
nasa_coordinates = [29.559684888503615, -95.0830971930759]
launch_site_map = folium.Map(location=nasa_coordinates, zoom_start=11, height='100%', width='100%')
launch_site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [56]:
# Create a blue circle at NASA Johnson Space Center's coordinate
circle = folium.Circle(nasa_coordinates, radius=1000, color='#0055dd', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
launch_site_map.add_child(circle)

# Create an icon showing an abbreviation of the name 'NASA Johnson Space Center'
marker = folium.map.Marker(nasa_coordinates, icon=DivIcon(icon_size=(10,10), icon_anchor=(-20,0), html='<div style="font-size: 16px; color:#000;"><b>%s</b></div>' % 'NASA_JSC'))
launch_site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


In [57]:
# List of coordinates and names for each location
site_coord = launch_sites_df[['Lat', 'Long']].values.tolist()
site_coord_name = list(launch_sites_df['Launch Site'])
site_coord_col = ['#32a852', '#e5ed47', '#ed47e8', '#f09816']

# Creating circles at each launch site
for index, i in enumerate(site_coord):
    circle_coordinates = i
    circle_popup_name = site_coord_name[index]
    circle_color = site_coord_col[index]

    # Circle
    circle = folium.Circle(circle_coordinates, radius=50, color=circle_color, fill=True).add_child(folium.Popup(circle_popup_name))
    launch_site_map.add_child(circle)

    # Marker
    marker_name = site_coord_name[index]
    marker = folium.map.Marker(
        circle_coordinates,
        icon=DivIcon(
            icon_size=(10,10),
            icon_anchor=(-20,0),
            html='<div style="font-size: 16px; color:#000;"><b>%s</b></div>' % marker_name
        )
    )
    launch_site_map.add_child(marker)

launch_site_map

## Observations

**Are all launch sites in proximity to the Equator line?**
These launch sites are not particularly close to the Equator (0° latitude). They're all in the northern hemisphere between 28° and 35° North latitude. The first three sites (CCAFS LC-40, CCAFS SLC-40, and KSC LC-39A) are all in Florida at around 28.5°N, while VAFB SLC-4E is in California at about 34.6°N.

**Are all launch sites in very close proximity to the coast?**
Yes, all of these launch sites are located very close to coastlines.
The three Florida sites (CCAFS and KSC) are located along the Atlantic coast at Cape Canaveral
The VAFB site (Vandenberg Air Force Base) is located on the Pacific coast in California

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [58]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [59]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [60]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

<ipython-input-60-5e608405af52>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)


,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [61]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame
for index, row in spacex_df.iterrows():
    # Create a custom icon for success/failure using HTML
    html = f'''
        <div style="width: 24px; height: 24px; border-radius: 12px; background-color: white;
             display: flex; justify-content: center; align-items: center; border: 2px solid {row['marker_color']}">
             <span style="color: {row['marker_color']}; font-weight: bold; font-size: 12px;">{index}</span>
        </div>
    '''

    # Create a custom DivIcon
    custom_icon = folium.DivIcon(
        icon_size=(24, 24),
        icon_anchor=(12, 12),
        html=html
    )

    # Create the marker with the custom icon
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],  # Or any other column you want to display
        icon=custom_icon
    )

    # Add marker to the marker cluster
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyse the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [70]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [71]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [72]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163

coastline_lat = 28.56367
coastline_lon = -80.57163
launch_site_lat = 28.563197
launch_site_lon = -80.576820


# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(distance_coastline)

0.5097431144955059


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [74]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property

# Coastline coordinates
coastline_lat = 28.56367
coastline_lon = -80.57163

# Create the marker
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

# Add the marker to the map
site_map.add_child(distance_marker)

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [75]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=1
)

# Add the PolyLine to the map
site_map.add_child(lines)

_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [76]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# Calculate the distance
city_lat = 28.07929
city_lon = -80.576820
launch_site_lat = 28.563197
launch_site_lon = -80.576820

distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)


In [77]:
# Create the marker
city_marker = folium.Marker(
    [city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    )
)

site_map.add_child(city_marker)

In [78]:
# Draw the line
city_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
    weight=1
)

site_map.add_child(city_line)

## Observations

**Are launch sites in close proximity to railways?**
Many SpaceX launch sites may be located near railways for logistical support, enabling the transportation of heavy materials and components. However, the exact proximity may vary by location.
Explanation: Railways provide an efficient means of transporting materials needed for launches. Being near rail infrastructure can be beneficial for supply chains.

**Are launch sites in close proximity to highways?**
Launch sites are likely to be situated within a reasonable distance from major highways to facilitate quick access for transporting personnel and equipment.
Explanation: Proximity to highways ensures that emergency services and support vehicles can reach the site quickly, which is essential for safety and operational efficiency.

**Are launch sites in close proximity to coastline?**
SpaceX launch sites are often near coastlines, particularly for ocean-bound launches. This allows for safer trajectories over water, minimizing risks to populated areas.
Explanation: Launching over the ocean reduces the danger to human populations and infrastructure in the event of a launch failure. Coastal proximity also provides an unobstructed view for tracking launches.

**Do launch sites keep certain distance away from cities?**
SpaceX launch sites typically maintain a significant distance from urban areas to mitigate risk and noise pollution.
Explanation: The risk of accidents during launches necessitates a buffer zone between launch sites and densely populated areas. Additionally, the noise and other disturbances from launches are less disruptive to communities when launch sites are situated further away.



## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
